# MAT281 - Tarea 1

## SVD y Gatitos 

En este ejercicio utilizaremos la descomposición valor singular con tal de comprimir imágenes, si bien hay algoritmos mucho mejores para esto, será un ejercicio muy ilustrativo. 

### Recuerdo

La descomposición SVD de una matriz $A$ de tamaño $m \times n$ posee la siguiente forma 
$$
A = U \Sigma V^H
$$
donde $\Sigma$ es diagonal $m \times n$, mientras que $U$ y $V$ son matrices unitarias $m \times m$ and $n \times n$, respectivamente. Los elementos diagonales de $\Sigma$ son no-negativos y aquellos valores positivos son llamados **valores singulares** de $A$. Como convención los valores singulares se listan en orden decreciente a lo largo de la diagonal. Las columnas de $U$ y $V$ son llamadas **vectores singulares** izquierdos y derechos respectivamente.

PD: Recuerda que $A^H = \bar{A}^\top$, es decir, la matriz traspuesta de la matriz conjugada.

In [ ]:
import numpy as np
from scipy.linalg import svd,diagsvd,norm

np.random.seed(42)  # Para reproducibilidad de resultados

Por ejemplo para una matriz de $2 \times 2$:

In [ ]:
A = np.array([[0.1, 0.5], [0.4, 0.8]])
u, s, vh = svd(A)
print(u)
print(s)
print(vh)

#### Ejercicio 1

(5 puntos)

Define la función `svd_validaton` tal que:

1. El _input_ sea un arreglo bidimensional `A` de tamaño $m \times n$.
2. Obtenga la descomposición valor singular de `A`.
3. Retorne `True` o `False` si es que se cumple la igualdad 
$$
A = \sum_{l=1}^{\min(m, n)} \sigma_l \; u_l v_l^H,
$$
donde $\sigma_l$ corresponden a los valores singulares de $A$, mientras que $u_i$ y $v_j$ a las columnas de $U$ y $V$ respectivamente. Hint: Utiliza `np.allclose` con la tolerancia por defecto.

In [ ]:
def svd_validation(A):
    u, s, vh = svd(A)
    suma=0
    for l in range(0,min(A.shape)):
        suma+= s[l]*(u[:,l].reshape(-1,1))*(vh[l,:].reshape(1,-1))# forma para escribir producto exterior, también ay alternativa documentación
    return np.allclose(A,suma)

In [ ]:
A_test = np.random.randint(100, size=(20, 5))
svd_validation(A_test)

### Geometría

Considerar una matriz `A` como un operador lineal tiene una interpretación geométrica muy sencilla, transforma una (hyper)-esfera en una (hyper)-elipse. Por ejemplo, consideremos una esfera unitaria en $\mathbb{R}^2$.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def show_circle(circle):
    plt.plot(circle[0, :], circle[1, :])
    plt.axis('image');

In [ ]:
t = np.linspace(0, 3.5 * np.pi , num=300)
l = np.linspace(-1, 1, num=10)
z = np.zeros_like(l)
circle = np.array(
    [
        np.concatenate([l, np.cos(t), z]),
        np.concatenate([z, np.sin(t), l])
    ]
)

show_circle(circle) 

Luego, al utilizar `A` como un operador lineal, es decir $A C$.

In [ ]:
print(A)

In [ ]:
show_circle(A @ circle)

#### Ejercicio 2

(5 puntos)

* Obtén la descomposición SVD de $A$, tal que $A = U \Sigma V^H$.
* Grafica el resultado de aplicar los siguientes operadores lineales a la circunferencia unitaria:
    - $U$
    - $\Sigma$
    - $V^H$
* Explica con tus palabras la transformación de la circunferencia unitaria luego de aplicar los operadores anteriores, ¿Influye en algo que $U$ y $V$ sean unitarias?
* ¿Qué relación tienen los operadores anteriores respecto a aplicar el operador lineal $A$?

In [ ]:
u, s, vh =  svd(A)

In [ ]:
show_circle(u @ circle)

In [ ]:
show_circle(diagsvd(s,A.shape[0],A.shape[1]) @ circle) #escribir matrix explicitamente o np.diag

In [ ]:
show_circle(vh @ circle)

__Respuesta:__ cada una de las aplicaciones corresponde a una rotación, esto esta relacionado con que en $\mathbb{R}^2$ las transformaciones unitarias son rotaciones o reflexiones. 

__Respuesta:__ El operador $U$ llevaría el circulo a otro espacio a través de rotaciones o reflexiones, preservando la distancia/angulos, luego el operador $\Sigma$ expandería o comprimiría en la direción de cada eje siendo en esta instancia que se producen cambios en las distancias/angulos de la figura, para al final , a través de $V^H$ volver a rotar/reflectar, sin perturbar distancias/angulos. Se estaría separando la transformación lineal en operaciones que conservan ángulos/distancias y operaciones lineales "simples" que cambian las dimensiones de la matriz(son matrices diagonales, en donde es fácil operar).

### Aproximación Rango Menor

Existen muchas maneras de expresar una matriz como una suma de matrices de menor rango, por ejemplo:

$$
\begin{bmatrix}
a & b \\
c & d 
\end{bmatrix}
= 
\begin{bmatrix}
a & 0 \\ 0 & 0  
\end{bmatrix}
+ 
\begin{bmatrix}
0 & b \\ 0 & 0 
\end{bmatrix}
+ 
\begin{bmatrix}
0 & 0 \\ c & 0 
\end{bmatrix}
+
\begin{bmatrix}
0 & 0 \\ 0 & d 
\end{bmatrix}.
$$

Cada una de las matrices del lado derecho pueden tener rango a lo más 1.

En el primer ejercicio demostraste que la descomposición SVD $A$ puede ser expresada como una suma de matrices. Sin embargo, cada una de estas matrices tiene rango 1! Esto pues cada una de estas matrices se forma a partir de los productos externos entre los vectores $u_l$ y $v_l$, es decir $u_l v_l^H$.

La pregunta natural es:

_¿Cómo obtener una buena aproximación de $A$ utilizando matrices de rango muy bajo?_

Bueno, aquí va un teorema.

__Teorema 1:__



Sea $A$ matriz de tamaño $m \times n$. Para cualquier $0 \le \ell \le r = \text{rank}(A)$, se define la matriz 
$$
A_\ell = \sum_{j=1}^{\ell} \sigma_j u_j v_j^*,
$$
utilizando los valores singulares $\sigma_j$ y los vectores singulares (izquierdos y derechos) $u_j, v_j$ de $A$,  i.e., $A_\ell$ se compone de la suma de los primeros $\ell$ términos de la descomposición SVD escrita como una suma de productor externos. Luego, el mínimo de  $\| A - B \|_F$ sobre todas las matrices $B$ de tamaño $m \times n$ y rango no mayor a $\ell$ se obtiene por $\| A - A_\ell \|_F$ y el mínimo que se alcanza es  $(\sigma_{\ell+1}^2 + \cdots + \sigma_r^2)^{1/2}$.

Recuerda que la norma de Frobenius se define como

$$
\| A \|_F = \bigg( \sum_{i, j} |A_{ij}|^2 \bigg)^{1/2}.
$$

Motivaremos el ejercicio utilizando imágenes en escala de grises ya que es muy intuitivo, fácil de ver y se puede considerar que la imagen es una matriz.

In [ ]:
from pathlib import Path
from PIL import Image

In [ ]:
# Utilizaremos una foto de una de mis gatitas c:
cat = Image.open(Path().resolve().parent / "images" / "coyoya.jpg").convert('L')
cat

Para convertirla en un numpy array basta con:

In [ ]:
cat_np = np.array(cat)
print(cat_np.shape)

#### Ejercicio 3

(10 puntos)

Define la función `low_rank_svd_approximation` tal que:

* Los inputs sean $A$ (la imagen convertida un `np.array` de dimensión 2) y un valor entero $\ell$ que represente la cantidad de términos a sumar de la despomposición SVD (respecto al teorema anterior).
* Retorne la aproximación $A_\ell$
* Imprima el error de la aproximación utilizando la norma de Frobenius.

Luego prueba tu función y observa la imagen con distintos valores de $\ell$.

In [ ]:
def low_rank_svd_approximation(A, l):
    u, s, vh = svd(A)
    suma=0
    for i in range(0,l):
        suma+= s[i]*np.outer(u[:,i],vh[i,:])    #(u[:,i].reshape(-1,1))*(vh[i,:].reshape(1,-1))
    print(norm(A-suma,'fro'))
    return suma    

In [ ]:
cat10 = low_rank_svd_approximation(cat_np, l=10)
plt.imshow(cat10, cmap='gray');

In [ ]:
cat50 = low_rank_svd_approximation(cat_np, l=50)

plt.imshow(cat50, cmap='gray');

#### Ejercicio 4

(15 puntos)

En el ejercicio anterior fijaste un rango máximo y obtuviste la aproximación, sin embargo, en otro contexto, te gustaría fijar una tolerancia de error y obtener la mejor aproximación.

Define la función `low_rank_svd_tol_approximation` tal que:

* Los inputs sean $A$ (la imagen convertida un `np.array` de dimensión 2) y $\varepsilon$ (tolerancia relativa) tal que
$$
\left(\frac{\sigma_{\ell+1}^2 + \cdots + \sigma_r^2}{\sigma_1^2 + \cdots + \sigma_r^2}\right)^{1/2} \le \varepsilon.
$$
* Imprima $\ell(\varepsilon)$, es decir, el mayor rango aproximado de $A$ tal que el error de aproximación sea a lo más $\varepsilon$.
* Retorne la aproximación $A_{\ell(\varepsilon)}$

Luego prueba tu función y observa la imagen con distintos valores de $\varepsilon$.

In [ ]:
def low_rank_svd_tol_approximation(A, tol):
    u, s, vh = svd(A)
    rango=min(A.shape)
    norma=norm(s)
    for l in range(0, rango): 
        if norm(s[l:])/norma <= tol:
            print(l)
            return low_rank_svd_approximation(A, l)
        

In [ ]:
cat_e1 = low_rank_svd_tol_approximation(cat_np, tol=1.e-1)
plt.imshow(cat_e1, cmap='gray');

In [ ]:
cat_e2 = low_rank_svd_tol_approximation(cat_np, tol=1.e-2)
plt.imshow(cat_e2, cmap='gray');

#### Ejercicio 5

(5 puntos)

Utilizando alguna imagen de tu preferencia utiliza ambas

In [ ]:
your_img_name = "negro.jpg"
your_img = Image.open(Path().resolve().parent / "images" / your_img_name ).convert('L')
your_img

In [ ]:
your_img_np = np.array(your_img)

In [ ]:
your_img10 = low_rank_svd_approximation(your_img_np, l=10)
plt.imshow(your_img10, cmap='gray');

In [ ]:
your_img50 = low_rank_svd_approximation(your_img_np, l=20)
plt.imshow(your_img50, cmap='gray');

In [ ]:
your_img_e1 = low_rank_svd_tol_approximation(your_img_np, tol=1.e-1)
plt.imshow(your_img_e1, cmap='gray');

In [ ]:
your_img_e2 = low_rank_svd_tol_approximation(your_img_np, tol=1.e-2)
plt.imshow(your_img_e2, cmap='gray');

In [ ]:
your_img10 = low_rank_svd_approximation(your_img_np, l=225)
plt.imshow(your_img10, cmap='gray');

__Pregunta:__ ¿Será una manera útil de comprimir imágenes en el disco duro o crees que existen otras formas más eficientes?

__Respuesta:__ Puede ser un paso dentro de un proceso más grande, pero por si sola, no sería suficiente. Si bien reduce la distancia en norma, una imagen donde solo hay un color tuvo problemas al ser comprimido por que no hay "entendimiento" de la imagen, teniendo en cuenta el posible problema de los repetidos, teniendo en cuenta que partes de la imagen requieren más detalle, y que parte solo poseen ruido,y  además teniendo en cuenta que si se aplicará directamente a cada canal no se tendría en cuenta correlación, entonces tal vez otros metodos que logren mejor "entendimiento" de la imagen sean más eficientes. Tal vez aplicarlo en ciertas zonas, para la luminosidad, y tratar el color por separado. Pero de todas formas pareciera no ser tan robusto(por ej: imagen solo negro capta ruido y pierde lo orginal), tal vez hayan métodos que logren resultados, incluso sin entendimiento.

## COVID-19 en Chile

### *¿Cuál es el panorama actual de Chile frente a la pandemia de COVID-19?*

In [ ]:
import pandas as pd
from datetime import date

#### Ejercicio 6

(25 puntos)

Se ha hablado mucho últimamente que Magallanes es una región crítica en cuanto a casos confirmados de COVID-19. Este ejercicio busca constatar a través de los datos aquellas aseveraciones con un indicador siemple pero que parece ser efectivo que bautizaremos como _tasa promedio de casos nuevos_, definida como el promedio de nuevos casos cada cien mil habitantes para un umbral de tiempo determinado.

Utiliza el dataframe `covid_comunas` para obtener el dataframe `covid_tasa_promedio_nuevos` que posee las columnas `region`, `comuna`, `promedio_casos_nuevos`, `poblacion` y `tasa_promedio_casos_nuevos` considerando el umbral de tiempo es entre el 1 y 11 de octubre del año 2020.

Para ello considera lo siguiente:

* No consideres registros que tengan código de comuna nulo.
* Rellena todos los registros de casos totales nulos por cero.
* Considera utilizar `melt` u otro método similar para apilar las columnas de fechas particulares en solo dos columnas, `fecha` y `casos_totales`.
* Define la columna `casos_nuevos` como la diferencia entre dos registros consecutivos para una misma comuna
    - No olvides ordenar por fecha
    - El primer registro de cada comuna debe ser nulo.
    - Considera utilizar el método `transform`.
* Filtra por el umbral de tiempo dado.
* Agrupa por región-comuna y luego define la columna `promedio_casos_nuevos` como el promedio de la columna `casos_nuevos`.
* En caso que hayas _dropeado_ la columna `poblacion` la puedes volver a agregar utilizando `merge` u otro método apropiado. Se asume que la población no cambia durante el tiempo.
* Asigna la columna `tasa_promedio_casos_nuevos` como la cantidad promedio de casos nuevos por cada cien mil habitantes.
* Ordena el dataframe resultante por `tasa_promedio_casos_nuevos` de manera descendente.

Información del dataset: [aquí](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto1).

In [ ]:
covid_comunas = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
)
covid_comunas.head()

In [ ]:
start_date = "2020-10-01" 
end_date = "2020-10-11"

In [ ]:
covid_comunas.dropna(subset=["codigo_comuna"]).drop(["codigo_region","tasa"],axis=1).info() #para notar donde hay nulos

In [ ]:
def nuevos_casos(s):
        n=s.copy()
        n.iloc[0]= np.nan
        n.iloc[1:]=s.iloc[1:].values-s.iloc[:-1].values
        return n 

covid_tasa_promedio_nuevos = (covid_comunas.dropna(subset=["codigo_comuna"]).drop(["codigo_region","tasa"],axis=1)
    .fillna(0) #se aplica a todas partes por facilidad teniendo en cuenta que no hay nulls ni en region ni en comuna ni en codigo_comuna
    .melt(id_vars=["region","comuna","codigo_comuna","poblacion"], var_name= "fecha" , value_name= "casos_totales"  )
    .sort_values(by=["codigo_comuna","fecha"]) 
    .assign(casos_nuevos=lambda df: df.groupby("comuna")["casos_totales"].transform(nuevos_casos) )
    [lambda df:(start_date <= df["fecha"]) & (df["fecha"]<= end_date)]
    .assign(tasa_promedio_casos_nuevos=lambda df: df["casos_nuevos"].mean()/df["poblacion"]*100000 ) 
    .sort_values(by="tasa_promedio_casos_nuevos",ascending=False)
    .drop(columns=["fecha","casos_totales","casos_nuevos"])
    .drop_duplicates()
)    


In [ ]:
covid_tasa_promedio_nuevos

__Pregunta:__ ¿Qué puedes observar respecto a las comunas que tienen mayor Tasa Promedio de Casos Nuevos?

__Respuesta:__

#### Ejercicio 7

(15 puntos)

¿Hay correlación entre la cantidad de exámenes PCR y los casos confirmados en cada comuna?


Información del dataset: [aquí](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto7).

In [ ]:
covid_pcr = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
)
covid_pcr

Obtén el dataframe `covid_pcr_melt` utilizando `covid_pcr` tal que:

* Tenga las columnas `region`, `fecha` y `nm_pcr`.
* `fecha` sea del tipo `datetime64`.
* `nm_pcr` sea el número de PCR realizados y rellena los valores nulos por cero.

In [ ]:
covid_pcr_melt = (
    covid_pcr.drop(["codigo_region","poblacion"],axis=1).melt(id_vars="region", var_name= "fecha" , value_name= "nm_pcr").fillna(0)
)
covid_pcr_melt["fecha"]=pd.to_datetime(covid_pcr_melt["fecha"],format='%Y-%m-%d')
covid_pcr_melt

Obtén el dataframe `covid_casos_melt` utilizando `covid_casos` tal que:

* Tenga las columnas `region`, `fecha` y `casos_totales`.
* Rellena los valores nulos con cero.
* `fecha` sea del tipo `datetime64`.
* `casos_totales` sea la cantidad de casos totales por región y fecha.

In [ ]:
covid_casos_melt = (
    covid_comunas.drop(["codigo_region","comuna","codigo_comuna","poblacion","tasa"],axis=1)
    .melt(id_vars="region", var_name= "fecha" , value_name= "casos_totales")
    .fillna(0)
    .groupby(["region","fecha"])
    .agg(casos_totales=("casos_totales","sum"))
    .reset_index()
)
covid_casos_melt["fecha"]=pd.to_datetime(covid_casos_melt["fecha"],format='%Y-%m-%d')
covid_casos_melt.info()

Finalmente, obtén la serie `covid_corr_casos_pcr` de la siguiente manera:

* Une `covid_pcr_melt` y `covid_casos_melt` a través de _merge_, utilizando la región y la fecha como llave, además conserva todos los registros (tanto derecha como izquierda).
* Rellena los números de PCR con el valor cero.
* Haz un `ffill` a los casos totales.
* Agrupa por región y obtén la correlación entre `nm_pcr` y `casos_totales`.
* Ordena los valores ascendentemente.

In [ ]:
covid_corr_casos_pcr = (
    covid_pcr_melt.merge(covid_casos_melt,how="outer",on=["region","fecha"]) )
covid_corr_casos_pcr["nm_pcr"]=covid_corr_casos_pcr["nm_pcr"].fillna(0)
covid_corr_casos_pcr["casos_totales"]=covid_corr_casos_pcr["casos_totales"].ffill()
covid_corr_casos_pcr= ( covid_corr_casos_pcr.ffill()
    .groupby("region")
    .apply(lambda df: df["nm_pcr"].corr(df["casos_totales"]))
    .sort_values()
)

covid_corr_casos_pcr

__Pregunta:__ ¿Qué puedes inferir del análisis anterior? ¿Se condice con tu sentido común?

__Respuesta:__ Existe algún grado de correlación, que es más débil o fuerte dependiendo de la región.Uno desearía que la cantidad de examenes pcr realizados fuera de la mano de los casos que hay en cada lugar, pero otros factores pueden afectar, como que tan saturado está la región y que tan fácil se podían obtener más examenes pcr. Sorprende también teniendo en cuenta la centralización en Chile.

#### Ejercicio 8

(10 puntos)

Propón y responde una pregunta que puedas resolver analizando dos o más conjuntos de datos del repositorio oficial de datos COVID-19 del Ministerio de Ciencia, Tecnología e Innovación de Chile ([link](https://github.com/MinCiencia/Datos-COVID19)).

Se evaluará originalidad, análisis de datos, calidad de la pregunta y respuesta.

__Pregunta:__ ¿Cúal es la relación entre cambios en la distribución etaria, el ratio de uso de ventiladores y el nivel de ocupación de residencias sanitarias?

Se importarán las bases de datos a utilizar:

In [ ]:
covid_residencias = ( 
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto36/ResidenciasSanitarias_std.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
) 
covid_residencias["fecha"]=pd.to_datetime(covid_residencias["fecha"],format='%Y-%m-%d')
display(covid_residencias)

In [ ]:
covid_ventiladores = ( 
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto20/NumeroVentiladores_T.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .rename(columns={"ventiladores":"fecha"})
) 
covid_ventiladores["fecha"]=pd.to_datetime(covid_ventiladores["fecha"],format='%Y-%m-%d')
display(covid_ventiladores)

In [ ]:
covid_genero_etario = ( 
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto16/CasosGeneroEtario_std.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
)
covid_genero_etario["fecha"]=pd.to_datetime(covid_genero_etario["fecha"],format='%Y-%m-%d')

display(covid_genero_etario)

Se pivotea el dataset covid_etario, y se crea uno nuevo que tenga en cuenta los casos nuevos diarios en el grupo etario

In [ ]:
def nuevos_casos(s):
        n=s.copy()
        n.iloc[0]= np.nan
        n.iloc[1:]=s.iloc[1:].values-s.iloc[:-1].values
        return n 

covid_etario=(covid_genero_etario
.groupby(["fecha","grupo_de_edad"])
.agg(casos=("casos_confirmados","sum"))
.reset_index()
# .assign(Total=lambda df: df.groupby("fecha").agg(lambda df: df[]))

 .pivot(index=["fecha"],columns="grupo_de_edad",values="casos")
 .reset_index()
 .assign(total=lambda df: df.iloc[:,1:].sum(axis=1))
)
display(covid_etario)
covid_etario_nuevo = covid_etario.copy()
covid_etario_nuevo.iloc[:,1:] = covid_etario.iloc[:,1:].agg(nuevos_casos) 

display(covid_etario_nuevo )

se pivotea el data set de residencias

In [ ]:
covid_residencias_pivot=(covid_residencias.groupby(["fecha","categoria"])
    .agg(numero=("numero","sum"))
    .reset_index()
    .pivot(index=["fecha"],columns="categoria",values="numero")
    .reset_index()
    .loc[lambda df: df["cupos totales"] != 0]
    )
display(covid_residencias_pivot)

se genera el ratio de casos en el grupo de edad respecto al total

In [ ]:
covid_etario_ratio=covid_etario.copy()
for i in covid_etario_ratio.columns[1:-1]:
    covid_etario_ratio[i]=(covid_etario[i]/covid_etario["total"] *100)
display(covid_etario_ratio)

covid_etario_nuevo_ratio=covid_etario_nuevo.copy()
for i in covid_etario_ratio.columns[1:-1]:
    covid_etario_nuevo_ratio[i]=(covid_etario_nuevo[i]/covid_etario_nuevo["total"] *100)
display(covid_etario_nuevo_ratio)

se obtiene el ratio de ocupación de residencias

In [ ]:
 covid_ratio_residencias= (covid_residencias_pivot.groupby("fecha")
                          .apply(lambda df: (df["usuarios en residencia"]/df["cupos totales"] ).values[0] ).to_frame().reset_index().rename(columns={0:"ratio_residencias"})
)
display(covid_ratio_residencias)

se obtiene el ratio de ocupación de ventiladores

In [ ]:
covid_ratio_ventiladores = covid_ventiladores.groupby("fecha").apply(lambda df: (df["ocupados"]/df["total"] ).values[0] ).to_frame().reset_index().rename(columns={0:"ratio_ventiladores"})
# covid_ventiladores.columns
display(covid_ratio_ventiladores)

se unen los datasets

In [ ]:
covid_merged_total= covid_ratio_ventiladores.merge(covid_ventiladores.drop(columns=["total","disponibles"],axis=1),how="inner",on="fecha").merge(covid_ratio_residencias,how="inner",on="fecha").merge(covid_etario,on="fecha")

covid_merged_nuevos= ( covid_ventiladores.drop(columns=["total","disponibles"],axis=1)
                     .merge(covid_ratio_ventiladores,how="inner",on="fecha")
                     .merge(covid_ratio_residencias,how="inner",on="fecha")
                     .merge(covid_etario_nuevo,on="fecha")
)

covid_merged_ratio= covid_ratio_ventiladores.merge(covid_ventiladores.drop(columns=["total","disponibles"],axis=1),how="inner",on="fecha").merge(covid_ratio_residencias,how="inner",on="fecha").merge(covid_etario_ratio,on="fecha")

covid_merged_nuevo_ratio= ( covid_ventiladores.drop(columns=["total","disponibles"],axis=1)
                     .merge(covid_ratio_ventiladores,how="inner",on="fecha")
                     .merge(covid_ratio_residencias,how="inner",on="fecha")
                     .merge(covid_etario_nuevo_ratio,on="fecha")
)

Se obtiene la correlación entre columnas

In [ ]:
correlation=covid_merged_total.corr()
correlation_nuevos=covid_merged_nuevos.corr()
correlation_ratio=covid_merged_ratio.corr()
correlation_nuevo_ratio=covid_merged_nuevo_ratio.corr()

veamos que pasa con la correlación entre los casos de distintos grupos etarios y la cantidad de ventiladores, el ratio de uso de ventiladores y el ratio de ocupación de residencias.

In [ ]:
display(correlation.iloc[0:3].T)

vemos que no hay variación significativa entre las correlaciones en lo subgrupos y la correlación del total de casos. La correlaciones serían positiva pero no tan fuerte entre el ratio de ocupación de residencias y los casos,  negativa entre la cantidad de casos y la cantidad de ventiladores ocupados, lo mismo para el ratio de ocupación de ventiladores. Pareciera ser que estamos obteniendo información de como han progresado la saturación de residencias/ventiladores con el numero de casos.

Notemos como es la correlación entre los casos en distintos grupos etarios

In [ ]:
display(correlation.iloc[3:,3:].T)

Fuertemente correlacionados, como se esperaría. 

Ahora veamos que pasa si tenemos en cuentas casos nuevos en vez de casos totales

In [ ]:
display(correlation_nuevos.iloc[0:3].T)

se pierde relación con la ocupación de residencias, pero se mantiene lo demás

¿Qué pasa al tener en cuenta el ratio de los casos en cada grupo etario con respecto al total?

Primero, veamos como el total de casos se relciona con los cambios de distribución etaria.

In [ ]:
display(correlation_ratio["total"].T)

Fuerte correlación negativa con adultos y positiva con adultos mayores. Es decir a medida que fue evolucionando la infección del covid en Chile, disminuyó la cantidad de adulto infectados respecto al total, pero aumentó la poroprción de jovenes y adulto mayores.

In [ ]:
display(correlation_ratio.iloc[:3].T)

Se identifican tres grupos: jóvenes, adultos y adultos mayores. para jovenes y adultos mayores la ocupación disminuye al aumentar su proporsión respecto al total de casos. Para adultos hay correlación fuertemente positiva sobretodo con respecto a la cantidad de ocupados.
Para el caso de la ocupación de residencias, notamos que no hay relación con respecto a infectados niños, si positiva para adultos jovenes,  mediana negativa para adultos y mediana positiva para adultos mayores. Para esto último, puede ser que lo que se esté manifestando es la relación del ratio de residencias con los infectados totales, y no tanto que tanto/como ocupan las residencias sanitarias cada grupo.

In [ ]:
display(correlation_ratio.iloc[3:,3:])

Al observar la correlación entre las columnas de los ratios, observamos el patrón de tres grupos.

Ahora si tenemos en cuenta el ratio de casos nuevos en cada grupo respecto a la cantidad de casos totales

In [ ]:
display(correlation_nuevo_ratio.iloc[:3].T)

Notamos que existe una relación positiva entre el ratio de ventidladores ocupados y la proporción de nuevos infectados adultos respecto al total. Se produce lo inverso para jovenenes, y la relación entre más una mayor cantidad de adultos mayores en los casos nuevos, es baja. 

__Respuesta:__ Pareciera ser que en las fases en donde hubo una mayor saturación del uso de ventiladores, estuvieron acompañadas de una mayor proporción de adultos contagiada. Especulando, pareciera ser que como la población adulta es una población que esta expuesta al virus al trabajar y mobilizarse(menor medida adultos mayores), padecen la enfermedad(no tanto para jovenes) y son una porción considerable de nuestra población,  influyen de manera importante en la ocupación de los ventiladores, siendo una porción considerable de la población de Chile, que puede ser que ocupe recursos críticos, como a su vez, puede ser que contagie a otros, que los ocuparán. También, es interesante que una mayor/menor poporción decasos nuevos respecto al total, parece no afectar al ratio de ventiladores usados, curioso. \
Para el caso de residencias sanitarias notamos que esta correlacionada con la pobación infectada, no de niños, que es lo esperable.\
Entre ratio de ocupación de ventiladores y de residencias sanitarias pareciera no haber relación.

__Extra__ intento de análisis con datasets por comuna.

In [ ]:
# covid_fallecidos = (
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo_std.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#     .astype({"fecha":"datetime64"})
    
# )

# covid_fallecidos_comuna = (
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna_std.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#     .astype({"fecha":"datetime64"})
#     .drop(["region","comuna"],axis=1)
# )
# display(covid_fallecidos_comuna)
# covid_fallecidos_comuna.info()
# covid_movilidad_comuna = (
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto33/IndiceDeMovilidad_std.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#     .astype({"fecha":"datetime64"})
#     .drop(["region","comuna"],axis=1)
# )
# display(covid_movilidad_comuna)
# covid_positividad_comuna = (
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto65/PositividadPorComuna_std.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#     .astype({"fecha":"datetime64"})
#     .drop(["region","comuna"],axis=1)
# )
# covid_positividad_comuna.info()
# display(covid_positividad_comuna)
# covid_BAC_comuna = ( # budqueda activa casos
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto64/BACPorComuna_std.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#     .astype({"fecha":"datetime64"})
#     .drop(["region","comuna"],axis=1)
# )
# display(covid_BAC_comuna)
# covid_cuarentena_comuna = ( 
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto29/Cuarentenas-Totales.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#      .astype({"fecha_de_inicio":"datetime64"})
#      .astype({"fecha_de_término":"datetime64"})
#     .drop(["region","n_region"],axis=1)
#     .rename(columns={"código_cut_comuna":"codigo_comuna"})
# ).loc[lambda df: df["estado"]=="Histórica"]


# display(covid_cuarentena_comuna)
# covid_activo_comuna = ( 
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna_std.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#     .astype({"fecha":"datetime64"})
#     .drop(["region","comuna"],axis=1)
# ) 
# display(covid_activo_comuna)
# covid_activo_comuna.poblacion.min()
# covid_cobertura_comuna = ( # budqueda activa casos
#     pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto66/CoberturaPorComuna_std.csv")
#     .rename(columns=lambda x: x.lower().replace(" ", "_"))
#     .dropna()
#     .astype({"fecha":"datetime64"})
#     .drop(["region","comuna"],axis=1)
# )
# display(covid_cobertura_comuna )


In [ ]:
# def nuevos_casos(s):
#         n=s.copy()
#         n.iloc[0]= np.nan
#         n.iloc[1:]=s.iloc[1:].values-s.iloc[:-1].values
#         return n 
# #     [lambda df:(start_date <= df["fecha"]) & (df["fecha"]<= end_date)]
# #     .assign(casos_nuevos=lambda df: df.groupby("comuna")["casos_totales"].transform(nuevos_casos) )
# covid_unido=(
# covid_activo_comuna.assign(casos_nuevos=lambda df: df.groupby("codigo_comuna")["casos_activos"].transform(nuevos_casos) )
#      .merge(covid_fallecidos_comuna.drop(["poblacion","codigo_region"],axis=1),how="inner",on=["codigo_comuna","fecha"])
#       .merge(covid_BAC_comuna.drop(["poblacion","codigo_region"],axis=1),how="inner",on=["codigo_comuna","fecha"])   
#       .merge(covid_cobertura_comuna.drop(["poblacion","codigo_region"],axis=1),how="inner",on=["codigo_comuna","fecha"])   
# #      .merge(covid_activo_comuna.drop(["codigo_region"],axis=1).assign(casos_nuevos=lambda df: df.groupby("codigo_comuna")["casos_activos"].transform(nuevos_casos) ).drop(["poblacion"],axis=1)    ,how="outer",on=["codigo_comuna","fecha"]) 
# #     .merge(covid_movilidad_comuna.drop(["poblacion","codigo_region"],axis=1).loc[lambda df: df["variable"]=="IM"].rename(columns={"value":"IM"}),how="inner",on=["codigo_comuna","fecha"])     
# #      .merge(covid_movilidad_comuna.drop(["poblacion","codigo_region"],axis=1).loc[lambda df: df["variable"]=="IM_interno"].rename(columns={"value":"IM_interno"}),how="inner",on=["codigo_comuna","fecha"]) 
# #      .merge(covid_movilidad_comuna.drop(["poblacion","codigo_region"],axis=1).loc[lambda df: df["variable"]=="IM_externo"].rename(columns={"value":"IM_externo"}),how="inner",on=["codigo_comuna","fecha"]) 
# #     .merge(covid_cuarentena_comuna.drop(["id","nombre","estado","alcance","detalle"],axis=1) , how="outer", on="codigo_comuna" )
#     .assign(tasa=lambda df: df["casos_nuevos"]/df["poblacion"]*100000 )
# )
# # covid_positividad_comuna.ffill()
# print(covid_unido["poblacion"].min())
# poblacion = pd.qcut(covid_unido["poblacion"], 10)
# grupo= (covid_unido.groupby(poblacion)
#      .apply(lambda df: df["cobertura_testeo"].corr(df["bac"])) #efectos cuarentena?
# )   
# display(grupo)
# identidad= lambda x:  x
# bins_dt = pd.date_range('2020-08-23', freq='W', periods=10)
# cycle = pd.cut(covid_unido["fecha"], bins_dt)
# display(cycle)
# # aaa=covid_unido.groupby([cycle, 'codigo_comuna']).agg({"IM_interno":"mean","tasa":"mean"}).reset_index()

# bins_dt = pd.date_range('2020-08-23', freq='W', periods=6)
# cycle = pd.cut(covid_BAC_comuna["fecha"], bins_dt)
# display(cycle)
# bbb=covid_BAC_comuna.groupby([cycle, 'codigo_comuna']).agg({"bac":"mean"}).reset_index()

# # bins_dt = pd.date_range('2020-08-23', freq='W', periods=10)
# # cycle = pd.cut(covid_positividad_comuna["fecha"], bins_dt)
# # display(cycle)
# # aaa=covid_positividad_comuna.groupby([cycle, 'codigo_comuna']).agg({"positividad":"mean"}).reset_index()
# display(covid_unido.groupby("fecha").apply(lambda df: df["tasa"].corr(df["cobertura_testeo"])) )

# display(grupo)

# display(covid_unido)
# covid_unido.describe()
# # covid_unido.plot()
# corr=lambda df:( (df["bac"]).corr(df["cobertura_testeo"]) )
# corr(covid_unido)
# # covid_fallecidos.groupby("region").apply(lambda df: df.plot(x="fecha",y="total"))

#### Ejercicio 9

(10 puntos)

Propón y responde una pregunta que puedas resolver analizando dos o más conjuntos de datos del repositorio oficial de datos COVID-19 del Ministerio de Ciencia, Tecnología e Innovación de Chile ([link](https://github.com/MinCiencia/Datos-COVID19)). Sin utilizar ninguno de los datasets que hayas utilizado en el ejercicio 8.

Se evaluará originalidad, análisis de datos, calidad de la pregunta y respuesta.

__Pregunta:__ ¿Cómo se relacionan el ratio de camas ocupadas y la positividad de los test PCR nivel regional a lo largo del tiempo?

Importaremos primero los datasets correspondientes

In [ ]:
covid_positividad_region = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto55/Positividad_por_region.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .dropna()
    .drop(["region_residencia","region"],axis=1)
)
covid_positividad_region["fecha"]=pd.to_datetime(covid_positividad_region["fecha"],format='%Y-%m-%d')
display(covid_positividad_region)

covid_camas_region = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto58/Camas_UCI_diarias_std.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .dropna()
    .rename(columns={"region":"codigo_region"})
)

covid_camas_region["fecha"]=pd.to_datetime(covid_camas_region["fecha"],format='%Y-%m-%d')

display(covid_camas_region)

Se cambia la columna de regiones por nombre a una de regiones por número

In [ ]:
covid_camas_region.replace({'Arica y Parinacota':15, 'Tarapacá':1, 'Antofagasta':2, 'Atacama' :3,'Coquimbo':4,
 'Valparaíso':5, 'Metropolitana':13, 'O’Higgins':6, 'Maule':7, 'Ñuble':16, 'Biobío':8,
 'Araucanía':9, 'Los Ríos':14, 'Los Lagos' :10,'Aysén':11, 'Magallanes':12},inplace=True)

Se obtiene el número de camas UCI ocupadas , respecto al total:

In [ ]:
covid_ratio_camas=(covid_camas_region.pivot(index=["codigo_region","fecha"],columns="serie",values="casos")
                   .reset_index()
                   .groupby(["codigo_region","fecha"])
                   .apply(lambda df:df["Camas UCI ocupadas"]/df["Camas UCI habilitadas"])
                   .reset_index().rename(columns={0:"ratio_camas"}).drop("level_2",axis=1) )
display(covid_ratio_camas)

se juntan las tablas

In [ ]:
covid_merged=covid_positividad_region.merge(covid_ratio_camas, how="inner", on=["codigo_region","fecha"]) 
display(covid_merged)

Se obtiene correlación entre las variables en cada instante de tiempo

In [ ]:
covid_corr_tiempo=covid_merged.groupby("fecha").apply(lambda df: df["positividad"].corr(df["ratio_camas"]))

Se gráfica para tener una idea

In [ ]:
covid_corr_tiempo.plot()

(parecen acciones jeje)

Se agrupan datos para tener una idea que pasa por región:

In [ ]:
cajitas = pd.date_range('2020-02-16', freq='SM', periods=16)
grupos = pd.cut(covid_merged["fecha"], cajitas)
corr_agrupado=covid_merged.groupby([grupos, 'codigo_region']).apply(lambda df: df["positividad"].corr(df["ratio_camas"])).reset_index().rename(columns={0:"correlacion_smensual"}).pivot(index=["fecha"],columns="codigo_region",values="correlacion_smensual").reset_index().drop(index=0)
display(corr_agrupado)
# display(corr_agrupado.corr() )  #descomente para obtener una idea de la relación(por métodos cuestionables)
# for i in corr_agrupado.columns[1:]:
#     corr_agrupado[i].plot(x="fecha",y=0) #sad, si quiere ignore esto :(

Pareciera que cada región sigue su camino, y al agrupar se nota el patrón 

Se obtiene una correlación en la positividad y el ratio de camas para comparar

In [ ]:
corr=lambda df:( (df["positividad"]).corr(df["ratio_camas"]) )
corr(covid_merged)

__Respuesta:__ Notamos que la correlación de todos lo datos no representa la situación completa. Notamos que en un intervalo de tiempo existión una correlación fuerte entre la positividad de los examenes PCR y la ocupación de camas, teniendo que ambas medidas de "saturación" estaban relacionadas a lo largo de Chile. LLuego, pasamos a una fase en que la correlación desaparece, la curva que se muestra puede estar relacionado a que pasamos de un colapso practicamente a nivel nacional, a aora a situaciones acotadas a cierta zonas.